In [1]:
import torch
import numpy as np
from torch import einsum
from tqdm.auto import tqdm
import seaborn as sns
from transformer_lens import HookedTransformer, ActivationCache, utils, patching
from datasets import load_dataset
from einops import einsum
import pandas as pd
from transformer_lens import utils
from rich.table import Table, Column
from rich import print as rprint
from jaxtyping import Float, Int, Bool
from typing import List, Tuple
from torch import Tensor
import einops
import functools
from transformer_lens.hook_points import HookPoint
# import circuitsvis
from IPython.display import HTML
from plotly.express import line
import plotly.express as px
from tqdm.auto import tqdm
import json
import gc
import plotly.graph_objects as go

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from plotly.subplots import make_subplots
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
pio.renderers.default = "notebook_connected"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

from haystack_utils import load_txt_data, get_mlp_activations, line
import haystack_utils

%reload_ext autoreload
%autoreload 2

In [2]:
news_data = haystack_utils.load_txt_data("german_news.csv")
print(len(news_data))

german_news.csv: Loaded 9246 examples with 0 to 22124 characters each.
9246


In [3]:
german_news_data = []
for example in tqdm(news_data[:-1]):
    index = example.index(";")
    example = example[index+1:]
    if len(example) > 500:
        german_news_data.append(example[:min(len(example), 2000)])

print(len(german_news_data))

  0%|          | 0/9245 [00:00<?, ?it/s]

8979


In [4]:
model = HookedTransformer.from_pretrained("EleutherAI/pythia-70m",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device=device)

german_data = haystack_utils.load_json_data("data/german_europarl.json")
english_data = haystack_utils.load_json_data("data/english_europarl.json")


english_activations = {}
german_activations = {}
for layer in range(3, 6):
    english_activations[layer] = get_mlp_activations(english_data[:200], layer, model, mean=False)
    german_activations[layer] = get_mlp_activations(german_data[:200], layer, model, mean=False)


LOG_PROB_THRESHOLD = -7
LAYER_TO_ABLATE = 3
NEURONS_TO_ABLATE = [669]
MEAN_ACTIVATION_ACTIVE = german_activations[LAYER_TO_ABLATE][:, NEURONS_TO_ABLATE].mean()
MEAN_ACTIVATION_INACTIVE = english_activations[LAYER_TO_ABLATE][:, NEURONS_TO_ABLATE].mean()

def deactivate_neurons_hook(value, hook):
    value[:, :, NEURONS_TO_ABLATE] = MEAN_ACTIVATION_INACTIVE
    return value
deactivate_neurons_fwd_hooks=[(f'blocks.{LAYER_TO_ABLATE}.mlp.hook_post', deactivate_neurons_hook)]

def activate_neurons_hook(value, hook):
    value[:, :, NEURONS_TO_ABLATE] = MEAN_ACTIVATION_ACTIVE
    return value
activate_neurons_fwd_hooks=[(f'blocks.{LAYER_TO_ABLATE}.mlp.hook_post', activate_neurons_hook)]

all_ignore, not_ignore = haystack_utils.get_weird_tokens(model, plot_norms=False)

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer
data/german_europarl.json: Loaded 2000 examples with 152 to 2000 characters each.
data/english_europarl.json: Loaded 2000 examples with 165 to 2000 characters each.


  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [89]:
german_neurons_l4 = [(4, 482), (4, 326)]

def deactivate_neurons_hook_l4(value, hook):
    value[:, :, german_neurons_l4] = english_activations[4][:, german_neurons_l4].mean()
    return value
deactivate_neurons_fwd_hooks_l4=[(f'blocks.{4}.mlp.hook_post', deactivate_neurons_hook_l4)]

def activate_neurons_hook_l4(value, hook):
    value[:, :, german_neurons_l4] = german_activations[4][:, german_neurons_l4].mean()
    return value
activate_neurons_fwd_hooks_l4=[(f'blocks.{4}.mlp.hook_post', activate_neurons_hook_l4)]


In [5]:
german_data = german_news_data

# Direct MLP5 effects

In [6]:
german_losses = []
for prompt in tqdm(german_data[:2000]):
    original_loss, ablated_loss, context_and_activated_loss, only_activated_loss = haystack_utils.get_direct_effect(prompt, model, pos=None, context_ablation_hooks=deactivate_neurons_fwd_hooks, context_activation_hooks=activate_neurons_fwd_hooks)
    german_losses.append((original_loss, ablated_loss, context_and_activated_loss, only_activated_loss))

  0%|          | 0/2000 [00:00<?, ?it/s]

In [7]:
def interest_measure(original_loss, ablated_loss, context_and_activated_loss, only_activated_loss):
    loss_diff = (ablated_loss - original_loss) # High ablation loss increase
    mlp_5_power = (only_activated_loss - original_loss) # Low loss increase from MLP5
    mlp_5_power[mlp_5_power < 0] = 0
    combined = 0.5*loss_diff - mlp_5_power
    combined[original_loss > 6] = 0
    combined[original_loss > ablated_loss] = 0
    return combined

In [8]:
def get_mlp5_decrease_measure(losses: list[tuple[Float[Tensor, "pos"], Float[Tensor, "pos"], Float[Tensor, "pos"], Float[Tensor, "pos"]]]):
    measure = []
    for original_loss, ablated_loss, context_and_activated_loss, only_activated_loss in losses:
        combined = interest_measure(original_loss, ablated_loss, context_and_activated_loss, only_activated_loss)
        measure.append(combined.max().item())
    return measure

measure = get_mlp5_decrease_measure(german_losses)
index = [i for i in range(len(measure))]

sorted_measure = list(zip(index, measure))
sorted_measure.sort(key=lambda x: x[1], reverse=True)

In [9]:
def print_prompt(prompt: str):
    str_token_prompt = model.to_str_tokens(model.to_tokens(prompt))
    original_loss, ablated_loss, context_and_activated_loss, only_activated_loss = haystack_utils.get_direct_effect(prompt, model, pos=None, context_ablation_hooks=deactivate_neurons_fwd_hooks, context_activation_hooks=activate_neurons_fwd_hooks)

    pos_wise_diff = interest_measure(original_loss, ablated_loss, context_and_activated_loss, only_activated_loss).flatten().cpu().tolist()

    loss_list = [loss.flatten().cpu().tolist() for loss in [original_loss, ablated_loss, context_and_activated_loss, only_activated_loss]]
    loss_names = ["original_loss", "ablated_loss", "context_and_activated_loss", "only_activated_loss"]
    haystack_utils.clean_print_strings_as_html(str_token_prompt[1:], pos_wise_diff, max_value=5, additional_measures=loss_list, additional_measure_names=loss_names)

In [191]:
def average_loss_plot(prompts: list[str], model: HookedTransformer, token="", plot=True):

    original_losses, ablated_losses, context_and_activated_losses, only_activated_losses = [], [], [], []
    names = ["Original", "Ablated", "Context + MLP5 active", "MLP5 active"]
    for prompt in prompts:
        original_loss, ablated_loss, context_and_activated_loss, only_activated_loss = haystack_utils.get_direct_effect(prompt, model, pos=-1, context_ablation_hooks=deactivate_neurons_fwd_hooks+deactivate_neurons_fwd_hooks_l4, context_activation_hooks=activate_neurons_fwd_hooks+activate_neurons_fwd_hooks_l4)
        original_losses.append(original_loss)
        ablated_losses.append(ablated_loss)
        context_and_activated_losses.append(context_and_activated_loss)
        only_activated_losses.append(only_activated_loss)
    if plot:
        haystack_utils.plot_barplot([original_losses, ablated_losses, context_and_activated_losses, only_activated_losses], names, ylabel="Loss", title=f"Average loss '{token}'")
    return original_losses, ablated_losses, context_and_activated_losses, only_activated_losses

## Looking for ngrams

In [144]:
for i, measure in sorted_measure[:10]:
    print_prompt(german_data[i])

In [104]:
# Ansicht
# Verteidig
# Minderheit

In [105]:
prompts = [
    "Der Handwerker wird einen maßgeschneiderten Schrank fertigen.",
    "Sie fertigt kunstvolle Keramikobjekte in ihrer Werkstatt.",
    "Der Architekt plant das Haus und der Bauunternehmer fertigt es.",
    "Das Unternehmen fertigt hochwertige Elektronikprodukte.",
    "Er hat eine Skulptur aus Metall fertiggestellt.",
    "Der Bäcker wird morgen früh das Brot fertigbacken.",
    "Die Designerin fertigt einzigartige Kleider für ihre Kunden.",
    "Der Ingenieur fertigt einen detaillierten Bauplan für das Projekt.",
    "Die Fabrik fertigt täglich tausende von Autos.",
    "Die Schneiderei fertigt individuelle Kleidungsstücke nach Maß."
]


for prompt in prompts:
    print_prompt(prompt)

## häufig

In [106]:
prompts = [
    "In der Sommerzeit kommt es häufig",
    "Der Zug hat häufig",
    "Eine gesunde Ernährung ist häufig",
    "Bei stressigen Situationen treten häufig",
    "Im Winter erkranken Menschen häufig",
    "In vielen Unternehmen werden häufig",
    "Eine gute Kommunikation ist häufig",
    "Um Verkehrsstaus zu vermeiden, fahren viele Pendler häufig"
]

for prompt in prompts:
    print_prompt(prompt)

In [107]:
average_loss_plot(prompts, model, token="hä-u-fig")

## Schließt

In [108]:
prompts = [
    "Bob, nachdem er alle Optionen gründlich erwogen hat, schließt",
    "Der Ausschuss hat die Angelegenheit gründlich erwogen und schließt",
    "Die Kommission hat eine wichtige Entscheidung getroffen und schließt",
    "Zusammen haben sie die Angelegenheit gründlich erwogen und schließt",
    "Die Polizei schließt"
]

for prompt in prompts:
    print_prompt(prompt)


In [109]:
average_loss_plot(prompts, model, token="sch-lie-ß-t")

## Beweglich

In [110]:
prompts = [
    "Zum Glück hat sich Klaus gut von seinem Unfall erholt und blieb geistig äußerst beweglich",
    "Das neue Design des Roboteres macht die Gliedmaßen besonders weit beweglich",
    "Die Roboterarme sind flexibel und beweglich",
    "Der Künstler hat eine Skulptur geschaffen, die optisch sehr beweglich",
    "Die Yoga-Übungen machen den Körper geschmeidig und beweglich",
    "Die Schubladen des Schranks sind dank der Rollen besonders beweglich",
    "Das neue Mobiltelefon ist leicht und beweglich",
    "Die Puppen sind sehr beweglich",
    "Die Beine vom Tisch sind beweglich",
    "Die Schmetterlinge fliegen anmutig und beweglich"
]



for prompt in prompts:
    print_prompt(prompt)

In [111]:
average_loss_plot(prompts, model, token="be-we-glich")

## Ansicht (not a good example)

In [112]:
prompts = [
    "Ich finde ihre Ansicht",
    "Ich teile seine Ansicht",
    "Ich teile deine Ansicht",
    "Die politische Ansicht",
    "Sie teilt die Ansicht",
    "Ich stimme mit deiner Ansicht"
]

for prompt in prompts:
    print_prompt(prompt)

In [113]:
average_loss_plot(prompts, model, token="Ans-icht")

## Voschlägen

In [114]:
# nächster
prompts = ["Ich habe noch einige Fragen zu Ihren Vorschlägen",
    "Ich stimme den Vorschlägen",
    "Kannst du bitte mit deinen Vorschlägen",
    "Laut den Vorschlägen",
    "Die Diskussion wurde mit vielen interessanten Vorschlägen",
    "Sie schrieb einen Brief mit ihren Vorschlägen",
    "Wir werden nach deinen Vorschlägen",
    "Sind Sie mit diesen Vorschlägen",
    "Gemäß den Vorschlägen",
    "Der Kunde war nicht einverstanden mit unseren Vorschlägen",
    "Sie zeigte uns ein Dokument mit ihren Vorschlägen",
    "Das Team arbeitet an den Vorschlägen",
    "Meinen Vorschlägen",
    "Der Ausschuss wird nach den Vorschlägen",
    "Sie war unzufrieden mit den Vorschlägen",
    "Unsere Agentur kam mit neuen Vorschlägen",
    "Haben Sie Änderungen zu den Vorschlägen",
    "Gemäß Ihren Vorschlägen",
    "Ich schrieb einen Bericht mit meinen Vorschlägen",
    "Nach den Vorschlägen",
    "Ich werde gemäß Ihren Vorschlägen",
    "Der Manager war unzufrieden mit den Vorschlägen",
    "Mit diesen Vorschlägen",
    "Der Ausschuss stimmte den Vorschlägen",
    "Der Leiter war sehr zufrieden mit den Vorschlägen",
    "Nach den aktuellen Vorschlägen",
    "Mit ihren innovativen Vorschlägen",
    "Mit einigen Verbesserungen zu Ihren Vorschlägen",
    "Das Team kam mit neuen Vorschlägen",
    "Sie kam mit großartigen Vorschlägen",
    "Ich werde mit meinen Vorschlägen",
    "Sie zeigte sich zufrieden mit den Vorschlägen",
    "Der Direktor war beeindruckt von den Vorschlägen",
    "Die Organisation hat nach Ihren Vorschlägen",
    "Wir haben ein Dokument mit den Vorschlägen",
    "Mit einigen Vorschlägen",
    "Der Lehrer war sehr zufrieden mit den Vorschlägen",
    "Nach Ihren Vorschlägen",
    "In Übereinstimmung mit den Vorschlägen",
    "Sie stimmte den Vorschlägen",
    "Mit diesen neuen Vorschlägen",
    "Ich bin sehr beeindruckt von Ihren Vorschlägen",
    "Das Unternehmen hat nach unseren Vorschlägen",
    "Die Jury war beeindruckt von den Vorschlägen",
    "Die Verwaltung hat nach Ihren Vorschlägen",
    "Das Publikum war sehr zufrieden mit den Vorschlägen",
]


for prompt in prompts[:5]:
    print_prompt(prompt)

In [115]:
average_loss_plot(prompts, model, token="V-orsch-lä-gen")

## Check previous tokens actually matter

### Generate some likely tokens from german data

In [116]:
token_counts = torch.zeros(model.cfg.d_vocab)
for example in tqdm(german_data):
    tokens = model.to_tokens(example)
    for token in tokens[0]:
        token_counts[token.item()] += 1

  0%|          | 0/8979 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [25]:
token_counts[all_ignore] = 0

punctuation = [".", ",", "!", "?", ";", ":", "-", "(", ")", "[", "]", "{", "}", "<", ">", "/", "\\", "\"", "'"]
leading_space_punctuation = [" " + char for char in punctuation]
punctuation_tokens = model.to_tokens(punctuation + leading_space_punctuation + [' –', " ", '  ', "<|endoftext|>"])[:, 1].flatten()
token_counts[punctuation_tokens] = 0

top_counts, top_tokens = torch.topk(token_counts, 100)
print(model.to_str_tokens(top_tokens[:100]))

['en', ' der', ' die', ' und', ' in', 'te', 'ung', 'ä', ' den', 't', 'z', 'st', 'ü', 'ch', ' er', ' von', 're', ' auf', ' mit', ' zu', 'ge', 'f', ' das', ' ein', ' K', ' im', 'ten', ' für', ' z', 'ig', 'e', ' W', ' sich', ' Die', 'it', 'er', 'hen', 'ö', 'ren', ' ist', ' an', ' ver', ' dem', 'k', 'in', ' nicht', ' des', ' w', 'i', 'w', 'ien', 'n', ' aus', 'l', 'h', ' Z', 'ß', ' eine', ' se', 'le', ' es', 'ischen', ' am', 'ag', ' G', 'et', 'ie', 's', 'ür', 'b', ' B', 'ht', 'es', ' auch', ' als', 'g', 'gen', 'we', 'men', ' D', ' d', 'hr', 'il', 'icht', ' F', ' sch', ' g', 'lich', ' nach', ' M', 'ungen', ' ge', ' um', ' über', ' be', 'ert', ' Ver', ' vor', ' we', ' Der']


In [198]:
def replace_token(prompts, replace_index, token="", num_replacements=20):
    new_prompts = []
    for prompt in prompts:
        tokens = model.to_tokens(prompt)
        for i in range(num_replacements):
            if tokens[0, replace_index].item() != top_tokens[i].item():
                new_tokens = tokens.clone()
                new_tokens[0, replace_index] = top_tokens[i]
                new_prompts.append(new_tokens)
            else:
                print("skipping")
    average_loss_plot(new_prompts, model, token=token)

def replace_token_loss(prompts, replace_index, num_replacements=10):
    new_prompts = []
    for prompt in prompts:
        tokens = model.to_tokens(prompt)
        for i in range(num_replacements):
            #if tokens[0, replace_index].item() != top_tokens[i].item():
            new_tokens = tokens.clone()
            new_tokens[0, replace_index] = top_tokens[i]
            new_prompts.append(new_tokens)
            #else:
            #    print("skipping")
    losses = []
    for prompt in new_prompts:    
        with model.hooks(fwd_hooks=activate_neurons_fwd_hooks+activate_neurons_fwd_hooks_l4):
            original_loss = model(prompt, return_type="loss", loss_per_token=True).flatten()[-1].item()
        losses.append(original_loss)
    return losses

In [117]:
prompts = ["Ich habe noch einige Fragen zu Ihren Vorschlägen",
    "Ich stimme den Vorschlägen",
    "Kannst du bitte mit deinen Vorschlägen",
    "Laut den Vorschlägen",
    "Die Diskussion wurde mit vielen interessanten Vorschlägen",
    "Sie schrieb einen Brief mit ihren Vorschlägen",
    "Wir werden nach deinen Vorschlägen",
    "Sind Sie mit diesen Vorschlägen",
    "Gemäß den Vorschlägen",
    "Der Kunde war nicht einverstanden mit unseren Vorschlägen",
    "Sie zeigte uns ein Dokument mit ihren Vorschlägen",
    "Das Team arbeitet an den Vorschlägen",
    "Meinen Vorschlägen",
    "Der Ausschuss wird nach den Vorschlägen",
    "Sie war unzufrieden mit den Vorschlägen",
    "Unsere Agentur kam mit neuen Vorschlägen",
    "Haben Sie Änderungen zu den Vorschlägen",
    "Gemäß Ihren Vorschlägen",
    "Ich schrieb einen Bericht mit meinen Vorschlägen",
    "Nach den Vorschlägen",
    "Ich werde gemäß Ihren Vorschlägen",
    "Der Manager war unzufrieden mit den Vorschlägen",
    "Mit diesen Vorschlägen",
    "Der Ausschuss stimmte den Vorschlägen",
    "Der Leiter war sehr zufrieden mit den Vorschlägen",
    "Nach den aktuellen Vorschlägen",
    "Mit ihren innovativen Vorschlägen",
    "Mit einigen Verbesserungen zu Ihren Vorschlägen",
    "Das Team kam mit neuen Vorschlägen",
    "Sie kam mit großartigen Vorschlägen",
    "Ich werde mit meinen Vorschlägen",
    "Sie zeigte sich zufrieden mit den Vorschlägen",
    "Der Direktor war beeindruckt von den Vorschlägen",
    "Die Organisation hat nach Ihren Vorschlägen",
    "Wir haben ein Dokument mit den Vorschlägen",
    "Mit einigen Vorschlägen",
    "Der Lehrer war sehr zufrieden mit den Vorschlägen",
    "Nach Ihren Vorschlägen",
    "In Übereinstimmung mit den Vorschlägen",
    "Sie stimmte den Vorschlägen",
    "Mit diesen neuen Vorschlägen",
    "Ich bin sehr beeindruckt von Ihren Vorschlägen",
    "Das Unternehmen hat nach unseren Vorschlägen",
    "Die Jury war beeindruckt von den Vorschlägen",
    "Die Verwaltung hat nach Ihren Vorschlägen",
    "Das Publikum war sehr zufrieden mit den Vorschlägen",
]

In [118]:
average_loss_plot(prompts, model, token="V-orsch-lä-gen")

In [119]:
# Replace second to last token
replace_token(prompts, -2, token="V-orsch-X-gen")

In [120]:
# Replace third to last token
replace_token(prompts, -3, token="V-X-lä-gen")

KeyboardInterrupt: 

In [192]:
# Replace V token
replace_token(prompts, -4, token="X-orsch-lä-gen")

# Compare with per-token chart below
replace_token([prompt], -4, token="X-orsch-lä-gen on single prompt")

In [ ]:
# Replace first token
replace_token(prompts, 1, token="X .... V-orsch-lä-gen")

In [199]:
prompt = "Ich habe noch einige Fragen zu Ihren Vorschlägen"
tokens = model.to_tokens(prompt)
losses = []
names = []

with model.hooks(fwd_hooks=activate_neurons_fwd_hooks+activate_neurons_fwd_hooks_l4):
    original_loss = model(prompt, return_type="loss", loss_per_token=True).flatten()[-1].item()
losses.append([original_loss]*20)
names.append("Original")
for pos in range(1, tokens.shape[1]-1):
    loss = replace_token_loss([prompt], pos, num_replacements=20)
    losses.append(loss)
    names.append(model.to_str_tokens(tokens[0, pos])[0])
haystack_utils.plot_barplot(losses, names, ylabel="Loss", title=f"Average loss")


In [141]:
new_prompts = ["Ich habe noch einige Fragen zu Ihren Vorschlägen"]

average_loss_plot(new_prompts, model, token="V-orsch-lä-gen")

In [140]:
new_prompts = [
    #" der die unden in enorschlägen"
    "Ich habe noch einige Fragen zu Ihren Vorsch dergen"
]

average_loss_plot(new_prompts, model, token="V-orsch-lä-gen")

## Weird behavior? Why is the loss so low...

In [145]:
prompt = "Kannst du bitte mit deinen Vorschlägen"
with model.hooks(fwd_hooks=activate_neurons_fwd_hooks+activate_neurons_fwd_hooks_l4):
    original_loss, cache = model.run_with_cache(prompt, return_type="loss", loss_per_token=True)

In [151]:
logit_attr_original, labels = haystack_utils.DLA([prompt], model)

  0%|          | 0/1 [00:00<?, ?it/s]

In [152]:
logit_attr_ablated , _ = haystack_utils.DLA(["Kannst du bitte mit deinenenorschlägen"], model)

  0%|          | 0/1 [00:00<?, ?it/s]

In [156]:
logit_diffs = (logit_attr_original - logit_attr_ablated).mean(0)
haystack_utils.line(logit_diffs.cpu().numpy(), xlabel="Correct logit", ylabel="", title="(Original DLA - Ablated DLA) per component", xticks=labels)

In [53]:
prompt = "Kannst du bitte mit deinen Vorschlägen"
loss = model(prompt, return_type="loss", loss_per_token=True).flatten()[-1]
print(loss)

tensor(3.2221, device='cuda:0')


In [54]:
prompt = "Kannst du bitte mit deinenenorschlägen"
loss = model(prompt, return_type="loss", loss_per_token=True).flatten()[-1]
print(loss)

tensor(0.4796, device='cuda:0')


In [172]:
prompt = " this that is die Vorschlägen"
loss = model(prompt, return_type="loss", loss_per_token=True).flatten()[-1]
print(loss)

tensor(2.4377, device='cuda:0')


In [67]:
prompt = " der die unden in enorschlägen"
loss = model(prompt, return_type="loss", loss_per_token=True).flatten()[-1]
print(loss)

tensor(0.3482, device='cuda:0')


In [87]:
prompt = " der die unden inen enorschlägen"
prob = model(prompt, return_type="logits").log_softmax(-1)
answer_token = model.to_single_token("gen")
print(prob[0, -2, answer_token])

tensor(-0.8953, device='cuda:0')


In [69]:
tokens = model.to_tokens(prompt)
pred_log_probs = model(prompt, return_type="logits").log_softmax(-1)

log_probs_for_predicted_tokens = prob[:, :-1].gather(dim=-1, index=tokens[:, 1:].unsqueeze(-1)).squeeze(-1)
log_probs_for_predicted_tokens

tensor([[-15.2951,  -6.8866, -13.0358,  -9.9525,  -7.3342, -14.4449,  -5.8242,
          -0.3482]], device='cuda:0')

In [57]:
prompt = "Kannst du bitte mit deinen Vorschlägen"
loss = model(prompt, return_type="loss", loss_per_token=True).flatten()[-1]
print_prompt(prompt)
print(loss)
prompt = " derannst du bitte mit deinen Vorschlägen"
loss = model(prompt, return_type="loss", loss_per_token=True).flatten()[-1]
print_prompt(prompt)
print(loss)

tensor(3.2221, device='cuda:0')


tensor(2.2443, device='cuda:0')


In [ ]:
def replace_token(prompts, replace_index, token="", num_replacements=20):
    new_prompts = []
    print(len(prompts))
    for prompt in prompts:
        tokens = model.to_tokens(prompt)
        for i in range(num_replacements):
            if tokens[0, replace_index].item() != top_tokens[i].item():
                new_tokens = tokens.clone()
                new_tokens[0, replace_index] = top_tokens[i]
                new_prompts.append(new_tokens)
            else:
                print("skipping")
    average_loss_plot(new_prompts, model, token=token)

# new_tokens[0, replace_index] = top_tokens[i]
            # if tokens[0, replace_index].item() != top_tokens[i].item():
            #else:
            #    print("skipping")

def replace_token_loss(prompts, replace_index, num_replacements=10):
    new_prompts = []
    for prompt in prompts:
        tokens = model.to_tokens(prompt)
        if tokens.shape[1] <= replace_index:
            continue
        for i in range(num_replacements):
            new_tokens = tokens.clone()
            new_tokens[0, replace_index] = top_tokens[i]
            new_prompts.append(new_tokens)

    losses = []
    for prompt in new_prompts:    
        with model.hooks(fwd_hooks=activate_neurons_fwd_hooks): # +activate_neurons_fwd_hooks_l4
            original_loss = model(prompt, return_type="loss", loss_per_token=True).flatten()[-1].item()
        losses.append(original_loss)
    
    return sum(losses) / len(losses)

# Compare with per-token chart below
replace_token([prompt], -4, token="X-orsch-lä-gen on single prompt")

prompt = "Ich habe noch einige Fragen zu Ihren Vorschlägen"
tokens = model.to_tokens(prompt)
losses = []
names = []

original_losses = []
for prompt in prompts:
    with model.hooks(fwd_hooks=activate_neurons_fwd_hooks+activate_neurons_fwd_hooks_l4):
        original_losses.append(model(prompt, return_type="loss", loss_per_token=True).flatten()[-1].item())
original_loss = sum(original_losses) / len(original_losses)    
losses.append([original_loss])
names.append("Original")

for pos in range(1, tokens.shape[1]-1):
    loss = replace_token_loss(prompts, pos, num_replacements=20)
    losses.append([loss])
    names.append(model.to_str_tokens(tokens[0, pos])[0])

haystack_utils.plot_barplot(losses, names, ylabel="Loss", title=f"Average loss")